In [1]:
import sqlite3
import pandas as pd

In [2]:
df = pd.read_csv('/content/WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [3]:
conn=sqlite3.connect('mytable.db')

In [4]:
df.to_sql(name="employee",con=conn,if_exists='replace',index=False)
conn.commit()

In [5]:
%load_ext sql
%sql sqlite:///mytable.db

In [6]:
query = "select * from employee where Age>35 AND EducationField = 'Medical'"
result_df = pd.read_sql_query(query, conn)
display(result_df)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,59,No,Travel_Rarely,1324,Research & Development,3,3,Medical,1,10,...,1,80,3,12,3,2,1,0,0,0
1,36,No,Travel_Rarely,1299,Research & Development,27,3,Medical,1,13,...,2,80,2,17,3,2,7,7,7,7
2,44,No,Travel_Rarely,477,Research & Development,7,4,Medical,1,36,...,4,80,1,24,4,3,22,6,5,17
3,43,No,Travel_Rarely,1273,Research & Development,2,2,Medical,1,46,...,4,80,2,6,3,2,5,3,1,4
4,50,No,Travel_Rarely,989,Research & Development,7,2,Medical,1,80,...,4,80,1,29,2,2,27,3,13,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,36,No,Travel_Rarely,557,Sales,3,3,Medical,1,2024,...,3,80,2,10,2,3,9,7,3,4
228,40,No,Travel_Rarely,1194,Research & Development,2,4,Medical,1,2051,...,2,80,3,20,2,3,5,3,0,2
229,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
230,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7


In [43]:
##Attrition Rate
query = """
SELECT
    COUNT(*),
    SUM(Attrition = 'Yes'),
    SUM(Attrition = 'Yes') * 100.0 / COUNT(*) AS AttritionRate
FROM employee;
"""
print("Overall Attrition Rate:\n", pd.read_sql(query, conn))

Overall Attrition Rate:
    COUNT(*)  SUM(Attrition = 'Yes')  AttritionRate
0      1470                     237      16.122449


In [8]:
##Attrition by Gender
query = """
SELECT
    Gender,
    COUNT(*),
    SUM(Attrition = 'Yes'),
    SUM(Attrition = 'Yes') * 100.0 / COUNT(*) AS AttritionRate
FROM employee
GROUP BY Gender;
"""
print("\nAttrition by Gender:\n", pd.read_sql(query, conn))


Attrition by Gender:
    Gender  COUNT(*)  SUM(Attrition = 'Yes')  AttritionRate
0  Female       588                      87      14.795918
1    Male       882                     150      17.006803


In [9]:
##Attrition by Department
query = """
SELECT
    Department,
    COUNT(*) AS Total,
    SUM(Attrition = 'Yes') AS Left,
    ROUND(SUM(Attrition = 'Yes') * 100.0 / COUNT(*), 2) AS AttritionRate
FROM employee
GROUP BY Department;
"""
print("\nAttrition by Department:\n", pd.read_sql(query, conn))


Attrition by Department:
                Department  Total  Left  AttritionRate
0         Human Resources     63    12          19.05
1  Research & Development    961   133          13.84
2                   Sales    446    92          20.63


In [10]:
##Attrition by Age
query = """
SELECT
  CASE
    WHEN Age < 26 THEN '18-25'
    WHEN Age < 36 THEN '26-35'
    WHEN Age < 46 THEN '36-45'
    WHEN Age < 56 THEN '46-55'
    ELSE '56+'
  END AS AgeGroup,

  COUNT(*) AS Total,

  SUM(Attrition = 'Yes') AS Left,

  ROUND(SUM(Attrition = 'Yes') * 100.0 / COUNT(*), 2) AS AttritionRate

FROM employee
GROUP BY AgeGroup
ORDER BY AgeGroup;
"""
print("\nAttrition by Age:\n", pd.read_sql(query, conn))


Attrition by Age:
   AgeGroup  Total  Left  AttritionRate
0    18-25    123    44          35.77
1    26-35    606   116          19.14
2    36-45    468    43           9.19
3    46-55    226    26          11.50
4      56+     47     8          17.02


In [31]:
##Attrition by YearsAtCompany
query = """
SELECT
    -- Group years into bins
    CASE
        When YearsAtCompany < 5 THEN '0-5'
        When YearsAtCompany >= 5 AND YearsAtCompany < 10 THEN '5-10'
        When YearsAtCompany >=10 AND YearsAtCompany < 15 THEN '10-15'
        When YearsAtCompany >=15 AND YearsAtCompany < 20 THEN '15-20'
        ELSE '20+'
    END AS YearsAtCompanyGroup,

    COUNT(*) AS Total,
    SUM(Attrition = 'Yes') AS Left,
    ROUND(SUM(Attrition = 'Yes') * 100.0 / COUNT(*), 2) AS AttritionRate
FROM employee
GROUP BY YearsAtCompanyGroup
ORDER BY YearsAtCompanyGroup;
"""
print("\nAttrition by Years at Company:\n", pd.read_sql(query, conn))


Attrition by Years at Company:
   YearsAtCompanyGroup  Total  Left  AttritionRate
0                 0-5    580   141          24.31
1               10-15    208    24          11.54
2               15-20     65     5           7.69
3                 20+     93     9           9.68
4                5-10    524    58          11.07


# **Why do more people over 50 years old leave the company than people who aged 40-50?**

In [12]:
query = """
SELECT
  CASE
    WHEN Age BETWEEN 40 AND 50 THEN '40-50'
    WHEN Age > 50 THEN '50+'
  END AS AgeGroup,
  COUNT(*) AS Total,
  SUM(Attrition = 'Yes') AS Left,
  ROUND(SUM(Attrition = 'Yes') * 100.0 / COUNT(*), 2) AS AttritionRate
FROM employee
WHERE Age >= 40
GROUP BY AgeGroup
ORDER BY AgeGroup;
"""
print("\n:\n", pd.read_sql(query, conn))


:
   AgeGroup  Total  Left  AttritionRate
0    40-50    379    39          10.29
1      50+    143    18          12.59


In [14]:
query = """
SELECT
  CASE
    WHEN Age BETWEEN 40 AND 50 THEN '40-50'
    WHEN Age > 50 THEN '50+'
  END AS AgeGroup,
  COUNT(*) AS TotalEmployees,
  SUM(Attrition = 'Yes') AS LeftEmployees,
  ROUND(SUM(Attrition = 'Yes') * 100.0 / COUNT(*), 2) AS AttritionRate
FROM employee
WHERE Age >= 40
GROUP BY AgeGroup;
"""
print("\n\n", pd.read_sql(query, conn))



   AgeGroup  TotalEmployees  LeftEmployees  AttritionRate
0    40-50             379             39          10.29
1      50+             143             18          12.59


In [25]:
query = """
SELECT
  OverTime,
  COUNT(*) AS Total,
  SUM(Attrition = 'Yes') AS Left,
  ROUND(SUM(Attrition = 'Yes') * 100.0 / COUNT(*), 2) AS AttritionRate
FROM employee
WHERE Age > 50
GROUP BY OverTime;
"""
print("\n\n", pd.read_sql(query, conn))



   OverTime  Total  Left  AttritionRate
0       No     99     9           9.09
1      Yes     44     9          20.45


In [24]:
query = """
SELECT
  OverTime,
  COUNT(*) AS Total,
  SUM(Attrition = 'Yes') AS Left,
  ROUND(SUM(Attrition = 'Yes') * 100.0 / COUNT(*), 2) AS AttritionRate
FROM employee
WHERE Age > 40 AND Age < 50
GROUP BY OverTime;
"""
print("\n\n", pd.read_sql(query, conn))



   OverTime  Total  Left  AttritionRate
0       No    205    15           7.32
1      Yes     87    14          16.09


After comparing the Attrition rates of 50+ and 40-50 age groups we see that the attrition rate of 50+ is more. This is because of many reasons one of the main one is Overtime. If we see aged people who are 50+ tend to leave if asked to work overtime compared to people of 40-50 age basket

# **Why do people with higher pay still leave the company?**

In [39]:
query = """
SELECT
  CASE
    WHEN MonthlyIncome < 3000 THEN 'Low'
    WHEN MonthlyIncome < 7000 THEN 'Mid'
    ELSE 'High'
  END AS IncomeGroup,
  COUNT(*) AS Total,
  SUM(Attrition = 'Yes') AS Left,
  ROUND(SUM(Attrition = 'Yes') * 100.0 / COUNT(*), 2) AS AttritionRate
FROM employee
GROUP BY IncomeGroup
ORDER BY AttritionRate DESC;
"""
print("\nAttrition by Income:\n", pd.read_sql(query, conn))



Attrition by Income:
   IncomeGroup  Total  Left  AttritionRate
0         Low    395   113          28.61
1         Mid    640    77          12.03
2        High    435    47          10.80


In [40]:
query = """
select
 OverTime,
 COUNT(*) AS Total,
 SUM(Attrition = 'Yes') AS Left,
 ROUND(SUM(Attrition = 'Yes') * 100.0 / COUNT(*), 2) AS AttritionRate
FROM employee
where MonthlyIncome >= 7000
GROUP BY OverTime;
"""
print("\n\n", pd.read_sql(query, conn))



   OverTime  Total  Left  AttritionRate
0       No    311    26           8.36
1      Yes    124    21          16.94


In [41]:
query = """
select
 WorkLifeBalance,
 COUNT(*) AS Total,
 SUM(Attrition = 'Yes') AS Left,
 ROUND(SUM(Attrition = 'Yes') * 100.0 / COUNT(*), 2) AS AttritionRate
FROM employee
where MonthlyIncome >= 7000
GROUP BY WorkLifeBalance;
"""
print("\n\n", pd.read_sql(query, conn))



    WorkLifeBalance  Total  Left  AttritionRate
0                1     19     3          15.79
1                2    104    13          12.50
2                3    264    27          10.23
3                4     48     4           8.33


In [42]:
query = """
select
 JobSatisfaction,
 COUNT(*) AS Total,
 SUM(Attrition = 'Yes') AS Left,
 ROUND(SUM(Attrition = 'Yes') * 100.0 / COUNT(*), 2) AS AttritionRate
FROM employee
where MonthlyIncome >= 7000
GROUP BY JobSatisfaction;
"""
print("\n\n", pd.read_sql(query, conn))



    JobSatisfaction  Total  Left  AttritionRate
0                1     89    14          15.73
1                2     85    11          12.94
2                3    125     9           7.20
3                4    136    13           9.56


People with higher pay still leave the company because of reasons like JobSatisfaction, WorkLifeBalance and Overtime which can be seen from above sql query outputs

# **Which factors drive employees who work at company less than 5 years to leave?**

In [34]:
query = """
SELECT
    -- Group years into bins
    CASE
        When YearsAtCompany < 5 THEN '0-5'
        When YearsAtCompany >= 5 AND YearsAtCompany < 10 THEN '5-10'
        When YearsAtCompany >=10 AND YearsAtCompany < 15 THEN '10-15'
        When YearsAtCompany >=15 AND YearsAtCompany < 20 THEN '15-20'
        ELSE '20+'
    END AS YearsAtCompanyGroup,

    COUNT(*) AS Total,
    SUM(Attrition = 'Yes') AS Left,
    ROUND(SUM(Attrition = 'Yes') * 100.0 / COUNT(*), 2) AS AttritionRate
FROM employee
GROUP BY YearsAtCompanyGroup
ORDER BY AttritionRate DESC;
"""
print("\nAttrition by Years at Company:\n", pd.read_sql(query, conn))


Attrition by Years at Company:
   YearsAtCompanyGroup  Total  Left  AttritionRate
0                 0-5    580   141          24.31
1               10-15    208    24          11.54
2                5-10    524    58          11.07
3                 20+     93     9           9.68
4               15-20     65     5           7.69


In [35]:
query = """
SELECT
 OverTime,
 COUNT(*) AS Total,
 SUM(Attrition = 'Yes') AS Left,
 ROUND(SUM(Attrition = 'Yes') * 100.0 / COUNT(*), 2) AS AttritionRate
FROM employee
where YearsAtCompany < 5
GROUP BY OverTime;
"""
print("\n\n", pd.read_sql(query, conn))



   OverTime  Total  Left  AttritionRate
0       No    406    66          16.26
1      Yes    174    75          43.10


In [38]:
query = """
SELECT
 WorkLifeBalance,
 COUNT(*) AS Total,
 SUM(Attrition = 'Yes') AS Left,
 ROUND(SUM(Attrition = 'Yes') * 100.0 / COUNT(*), 2) AS AttritionRate
FROM employee
where YearsAtCompany < 5
GROUP BY WorkLifeBalance
ORDER BY AttritionRate DESC;
"""
print("\n\n", pd.read_sql(query, conn))



    WorkLifeBalance  Total  Left  AttritionRate
0                1     35    14          40.00
1                4     54    16          29.63
2                2    134    35          26.12
3                3    357    76          21.29


If we see the attrition rates of people leaving organisation before 5 years, its the highest compared to others. Main reasons for that are OverTime and WorkLifeBalance as the attrition rate is more for these 2 factors for people leaving company before 5 years